In [14]:
# Examine the right order of different phase corrections in time-series: tropo, DEM error and/or ramp
# The simulated time-series includes:
#     1) simulated displacement time-series
#     2) stratified tropospheric delay from MERRA
#     3) stochatic tropospheric turbulence based on power-law characteristics
#        with power factor estimated from real time-series phase residual
#     4) topographic residual phase from DEM error estimated in real data
#     5) plane phase ramp estimated from real data
#
# The simulated time-series can be generated from the following notebook:
#     1) sim_trop_turbulence
#     2) sim_timeseries

import os

work_dir = '/Users/yunjunz/development/python/2018_ComputGeosci_PySAR/phaseCorrection_order'
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)
os.chdir(work_dir)
print('Go to directory: '+work_dir)


sim_vel_file = 'velocity_sim.h5'
pysar_vel_file = 'velocity_pysar.h5'
giant_vel_file = 'velocity_giant.h5'


Go to directory: /Users/yunjunz/development/python/2018_ComputGeosci_PySAR/phaseCorrection_order


In [15]:
# simulated velocity: truth
cmd = 'timeseries2velocity.py timeseriesSim.h5 -o {}'.format(sim_vel_file)
print(cmd)
os.system(cmd)

timeseries2velocity.py timeseriesSim.h5 -o velocity_sim.h5


0

In [5]:
# velocity with timeseries phase correction order from PYSAR
cmd = 'diff.py timeseries.h5 ECMWF.h5 -o timeseries_ECMWF.h5\n'
cmd += 'dem_error.py timeseries_ECMWF.h5 -g geometryRadar.h5 -o timeseries_ECMWF_demErr.h5\n'
cmd += 'remove_ramp.py timeseries_ECMWF_demErr.h5 -m waterMask.h5 -o '

256.0